# Парсинг сайтов Python
## bs4, структура запросов. User-agent и другие заголовки

In [1]:
import requests
from bs4 import BeautifulSoup
import lxml

In [2]:
header = {'user-agent': '****'}

In [3]:
link = 'https://browser-info.ru/'
data = requests.get(link, headers = header)
print(data.status_code)
print(data.text[:300])

200
<html>
    <head>
        <meta http-equiv="Content-Type" content="text/html; charset=utf-8">
        <title>Информация о вашем браузере онлайн. Узнать версию браузера, активные настройки, список плагинов.</title>
        <meta name="description" content="На этой странице вы можете узнать всю ин


In [4]:
with open('1.html', 'w', encoding='utf-8') as file:
    file.write(data.text)

In [5]:
soup = BeautifulSoup(data.text, 'lxml')

In [6]:
block = soup.find('div', id = 'tool_padding')
check_js = block.find('div', id = 'javascript_check')
check_js

<div id="javascript_check"><span class="option_title">Javascript</span> - <span class="off">выключено</span></div>

In [7]:
result = check_js.find_all('span')[1].text
result

'выключено'

In [8]:
res1 = f'javascript: {result}'

In [9]:
check_flash = block.find('div', id = 'flash_version')
status_flash = check_flash.find_all('span')
status_flash

[<span class="option_title">Flash:</span>,
 <span class="off">отсутствует/выключен</span>]

In [10]:
status_flash = status_flash[1].text
res2 = f'flash: {status_flash}'

In [11]:
check_user = block.find('div', id = 'user_agent').text
check_user

'User-agent: antih'

In [12]:
res3 = f'user-agent: {check_user}'

In [13]:
res1, res2, res3

('javascript: выключено',
 'flash: отсутствует/выключен',
 'user-agent: User-agent: antih')

### подменим user-agent

In [14]:
%pip install fake_useragent

Note: you may need to restart the kernel to use updated packages.


In [15]:
import fake_useragent

In [16]:
user = fake_useragent.UserAgent().random
header = {'user-agent': user}
link = 'https://browser-info.ru/'
data = requests.get(link, headers = header)
soup = BeautifulSoup(data.text, 'lxml')

In [17]:
check_user = soup.find('div', id = 'user_agent').text
check_user

'User-agent: Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1667.0 Safari/537.36'

# Авторизация на сайте, использование куки и сессий

In [18]:
session = requests.Session()

In [19]:
link = 'https://account.wolfram.com/login/oauth2/sign-in'
user = fake_useragent.UserAgent().random
header = {'user-agent': user}
data = {'user': '******',
        'password': '*****'}

In [20]:
response = session.post(link, data=data, headers=header)
response

<Response [200]>

In [21]:
link2 = 'https://wolfram.com/'
link2_resp = session.get(link2, headers=header).text

In [22]:
cookies_dict = [
    {"domain": key.domain, "name": key.name, "path": key.path, "value": key.value}
    for key in session.cookies
]

In [23]:
session2 = requests.Session()
for cookies in cookies_dict:
    session2.cookies.set(**cookies)

In [24]:
from IPython.core.display import display, HTML

resp = session2.get('https://wolfram.com/', headers=header)
display(HTML(resp.text))

#  Парсинг всего сайта и скачивание файлов внутри него

In [27]:
page = 1
image_number = 0
link =f'https://zastavok.net'

In [28]:
for i in range(172):
    response = requests.get(f'{link}/devushki/{page}', headers = header).text
    soup = BeautifulSoup(response, 'lxml')
    block = soup.find('div', class_ = 'block-photo')
    all_images = block.find_all('div', class_ = 'short_full')
    for image in all_images:
        image_link = image.find('a').get('href')
        download_link = requests.get(f'{link}/{image_link}').text
        download_soup = BeautifulSoup(download_link, 'lxml')
        block = download_soup.find('div', class_ = 'image_data').find('div', class_ = 'block_down')
        result_link = block.find('a').get('href')
        print(f'{link}{result_link}')
        image_source = requests.get(f'{link}{result_link}').content
        with open(f'images/{image_number}.jpg', 'wb') as file:
            file.write(image_source)
        image_number += 1
        print(f'image {image_number}.jpg saved!')
        break
    page += 1
    break

https://zastavok.net/download/60012/1920x1200/
image 1.jpg saved!


# Проверим список прокси сайтом http://icanhazip.com/

In [29]:
# import re

In [30]:
with open('proxy') as file:
    proxy_base = ''.join(file.readlines()).strip().replace('\t', '').split('\n')
proxy_base

['103.253.27.108:80',
 '91.89.89.5:8080',
 '191.101.39.81:80',
 '41.217.219.53:31398',
 '154.72.204.122:8080',
 '191.101.39.29:80',
 '191.101.39.2:80',
 '185.198.188.51:8080',
 '15.188.97.203:80',
 '185.198.188.52:8080',
 '118.174.233.45:44061']

In [31]:
for proxy in proxy_base:
    proxies = {
        'http': f'http://{proxy}',
        'https': f'http://{proxy}'
    }
    link = 'http://icanhazip.com/'
    try:
        print(f'Try {proxy}')
        response = requests.get(link, proxies=proxies, timeout=2, headers = header).text
        print(f'IP: {response} Well doneS')
    except:
        print(f'Proxy is broken')
    

Try 103.253.27.108:80
Proxy is broken
Try 91.89.89.5:8080
Proxy is broken
Try 191.101.39.81:80
Proxy is broken
Try 41.217.219.53:31398
Proxy is broken
Try 154.72.204.122:8080
Proxy is broken
Try 191.101.39.29:80
Proxy is broken
Try 191.101.39.2:80
Proxy is broken
Try 185.198.188.51:8080
Proxy is broken
Try 15.188.97.203:80
Proxy is broken
Try 185.198.188.52:8080
Proxy is broken
Try 118.174.233.45:44061
Proxy is broken


### Добавим распараллеливание

In [32]:
import multiprocessing
import pars

# для распараллеливания неоходимо поместить функцию в файл pars.py
# def handler(proxy):
#     proxies = {
#         'http': f'http://{proxy}',
#         'https': f'http://{proxy}'
#     }
#     link = 'http://icanhazip.com/'

#     try:
#         print(f'Try {proxy}')
#         response = requests.get(link, proxies=proxies, timeout=2).text
#         return (f'IP: {response} Well done')
#     except:
#         return (f'Proxy is broken')
        
with open('proxy') as file:
    proxy_base = ''.join(file.readlines()).strip().replace('\t', '').split('\n')

multiprocessing.Pool().map(pars.handler, proxy_base)

Try 103.253.27.108:80Try 91.89.89.5:8080Try 191.101.39.81:80Try 41.217.219.53:31398


Try 118.174.233.45:44061Try 154.72.204.122:8080

Try 191.101.39.2:80Try 185.198.188.51:8080Try 191.101.39.29:80Try 185.198.188.52:8080Try 15.188.97.203:80




In [33]:
pool = multiprocessing.Pool()
results = pool.map(pars.my_function, [1,2,3,4,5,6])
results